In [11]:
import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.model import Model

from qiskit import BasicAer
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.algorithms import CplexOptimizer, MinimumEigenOptimizer
from qiskit_optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp

from qiskit_optimization.converters import InequalityToEquality, IntegerToBinary, LinearEqualityToPenalty

<ipython-input-11-5c6fd6ede52b>:6: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms import QAOA, NumPyMinimumEigensolver


In [1]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --config python3
!python3 --version
!sudo apt install python3-pip

update-alternatives: error: alternative path /usr/bin/python3.8 doesn't exist
update-alternatives: error: no alternatives for python3
Python 3.10.12
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python3-pip python3-setuptools python3-wheel
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 1,677 kB of archives.
After this operation, 8,965 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-setuptools all 59.6.0-1.2ubuntu0.22.04.1 [339 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-wheel all 0.37.1-2ubuntu0.22.04.1 [32.0 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip all 22.0.2+dfsg-1ubuntu0.3 [1,305 kB]
F

In [12]:
import pandas as pd

df = pd.read_csv('box_data.csv').set_index('Box Type')

cargo_list = []
base_area_list = []
volume_list = []

# Iterate over each row
for index, rows in df.iterrows():
    # Create list for the current row
    my_list =[rows.Length, rows.Width, rows.Height, rows.Base_area, rows.Volume, rows.Weight]
    base_area = rows.Base_area
    volume = rows.Volume

    # append the list to the final list
    cargo_list.append(my_list)
    base_area_list.append(base_area)
    volume_list.append(volume)
    volume_list.sort(reverse=True)


# print list
print(cargo_list)

base_area_list.sort()
print(base_area_list)
df.head()
df.dtypes

[[12, 5, 5, 60, 300, 500], [23, 5, 5, 115, 575, 700], [5, 5, 5, 25, 125, 200]]
[25, 60, 115]


Length       int64
Width        int64
Height       int64
Base_area    int64
Volume       int64
Weight       int64
dtype: object

In [13]:
n = 2 # number of bins

m = len(cargo_list) # number of items

Q =  2000  # max base area

wj = np.amax(base_area_list) #  picking the maximum base area

container_volume = Q * 200

In [14]:
# Construct model using docplex
mdl = Model("BinPacking")

x = mdl.binary_var_list([f"x{i}" for i in range(n)]) # list of variables that represent the bins
e =  mdl.binary_var_list([f"e{i//m},{i%m}" for i in range(n*m)]) # variables that represent the items on the specific bin

objective = mdl.sum([x[i] for i in range(n)])

mdl.minimize(objective)

In [15]:
for j in range(m):
    # First set of constraints:
    constraint0 = mdl.sum([e[i*m+j] for i in range(n)])
    mdl.add_constraint(constraint0 == 1, f"cons0,{j}")

for i in range(n):
    # Second set of constraints: sum of box area < container area
    constraint1 = mdl.sum(base_area_list[i] for i in range(m))
    mdl.add_constraint(constraint1 <= Q, f"cons1,{i}")

for i in range(n):
		constraint2 = mdl.sum(volume_list[i] for i in range(m, m-3, -1))
		mdl.add_constraint(constraint2 <= container_volume, f"cons2,{i}")




IndexError: ignored

In [16]:
# convert from DOcplex model to Qiskit Quadratic program
# qp = QuadraticProgram()
qp = from_docplex_mp(mdl)
print(qp.export_as_lp_string())

# Solving Quadratic Program using CPLEX
qp = from_docplex_mp(mdl)

cplex = CplexOptimizer()
result = cplex.solve(qp)
print(result)
print(qp.prettyprint())
# plot_bins(result, wj, n, m)


<ipython-input-16-609b72b94edb>:3: UserWarning: Trivial constraint: cons1,0: 200 <= 2000
  qp = from_docplex_mp(mdl)
<ipython-input-16-609b72b94edb>:3: UserWarning: Trivial constraint: cons1,1: 200 <= 2000
  qp = from_docplex_mp(mdl)


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: BinPacking

Minimize
 obj: x0 + x1
Subject To
 cons0,0: _e0,0 + _e1,0 = 1
 cons0,1: _e0,1 + _e1,1 = 1
 cons0,2: _e0,2 + _e1,2 = 1
 cons1,0: <= 1800
 cons1,1: <= 1800

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= _e0,0 <= 1
 0 <= _e0,1 <= 1
 0 <= _e0,2 <= 1
 0 <= _e1,0 <= 1
 0 <= _e1,1 <= 1
 0 <= _e1,2 <= 1

Binaries
 x0 x1 _e0,0 _e0,1 _e0,2 _e1,0 _e1,1 _e1,2
End



<ipython-input-16-609b72b94edb>:7: UserWarning: Trivial constraint: cons1,0: 200 <= 2000
  qp = from_docplex_mp(mdl)
<ipython-input-16-609b72b94edb>:7: UserWarning: Trivial constraint: cons1,1: 200 <= 2000
  qp = from_docplex_mp(mdl)


fval=0.0, x0=0.0, x1=0.0, e0,0=1.0, e0,1=1.0, e0,2=1.0, e1,0=0.0, e1,1=0.0, e1,2=0.0, status=SUCCESS
Problem name: BinPacking

Minimize
  x0 + x1

Subject to
  Linear constraints (5)
    e0,0 + e1,0 == 1  'cons0,0'
    e0,1 + e1,1 == 1  'cons0,1'
    e0,2 + e1,2 == 1  'cons0,2'
    0 <= 1800  'cons1,0'
    0 <= 1800  'cons1,1'

  Binary variables (8)
    x0 x1 e0,0 e0,1 e0,2 e1,0 e1,1 e1,2



In [17]:
ineq2eq = InequalityToEquality()
qp_eq = ineq2eq.convert(qp)
print(qp_eq.export_as_lp_string())
print(f"The number of variables is {qp_eq.get_num_vars()}")

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: BinPacking

Minimize
 obj: x0 + x1
Subject To
 cons0,0: _e0,0 + _e1,0 = 1
 cons0,1: _e0,1 + _e1,1 = 1
 cons0,2: _e0,2 + _e1,2 = 1
 cons1,0: cons1,0@int_slack = 1800
 cons1,1: cons1,1@int_slack = 1800

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= _e0,0 <= 1
 0 <= _e0,1 <= 1
 0 <= _e0,2 <= 1
 0 <= _e1,0 <= 1
 0 <= _e1,1 <= 1
 0 <= _e1,2 <= 1
       cons1,0@int_slack <= 1800
       cons1,1@int_slack <= 1800

Binaries
 x0 x1 _e0,0 _e0,1 _e0,2 _e1,0 _e1,1 _e1,2

Generals
 cons1,0@int_slack cons1,1@int_slack
End

The number of variables is 10


In [18]:
int2bin = IntegerToBinary()
qp_eq_bin = int2bin.convert(qp_eq)
print(qp_eq_bin.export_as_lp_string())
print(f"The number of variables is {qp_eq_bin.get_num_vars()}")

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: BinPacking

Minimize
 obj: x0 + x1
Subject To
 cons0,0: _e0,0 + _e1,0 = 1
 cons0,1: _e0,1 + _e1,1 = 1
 cons0,2: _e0,2 + _e1,2 = 1
 cons1,0: cons1,0@int_slack@0 + 2 cons1,0@int_slack@1 + 4 cons1,0@int_slack@2
          + 8 cons1,0@int_slack@3 + 16 cons1,0@int_slack@4
          + 32 cons1,0@int_slack@5 + 64 cons1,0@int_slack@6
          + 128 cons1,0@int_slack@7 + 256 cons1,0@int_slack@8
          + 512 cons1,0@int_slack@9 + 777 cons1,0@int_slack@10 = 1800
 cons1,1: cons1,1@int_slack@0 + 2 cons1,1@int_slack@1 + 4 cons1,1@int_slack@2
          + 8 cons1,1@int_slack@3 + 16 cons1,1@int_slack@4
          + 32 cons1,1@int_slack@5 + 64 cons1,1@int_slack@6
          + 128 cons1,1@int_slack@7 + 256 cons1,1@int_slack@8
          + 512 cons1,1@int_slack@9 + 777 cons1,1@int_slack@10 = 1800

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= _e0,0 <= 1
 0 <= _e0,1 <= 1
 0 <= _e0,2 <= 1
 0 <= _e1,0 <= 1
 0 <= _e1,1 <= 1
 0 <= _e1,2 

In [19]:
lineq2penalty = LinearEqualityToPenalty()
qubo = lineq2penalty.convert(qp_eq_bin)
print(f"The number of variables is {qp_eq_bin.get_num_vars()}")
print(qubo.export_as_lp_string())

The number of variables is 30
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: BinPacking

Minimize
 obj: x0 + x1 - 6 _e0,0 - 6 _e0,1 - 6 _e0,2 - 6 _e1,0 - 6 _e1,1 - 6 _e1,2
      - 10800 cons1,0@int_slack@0 - 21600 cons1,0@int_slack@1
      - 43200 cons1,0@int_slack@2 - 86400 cons1,0@int_slack@3
      - 172800 cons1,0@int_slack@4 - 345600 cons1,0@int_slack@5
      - 691200 cons1,0@int_slack@6 - 1382400 cons1,0@int_slack@7
      - 2764800 cons1,0@int_slack@8 - 5529600 cons1,0@int_slack@9
      - 8391600 cons1,0@int_slack@10 - 10800 cons1,1@int_slack@0
      - 21600 cons1,1@int_slack@1 - 43200 cons1,1@int_slack@2
      - 86400 cons1,1@int_slack@3 - 172800 cons1,1@int_slack@4
      - 345600 cons1,1@int_slack@5 - 691200 cons1,1@int_slack@6
      - 1382400 cons1,1@int_slack@7 - 2764800 cons1,1@int_slack@8
      - 5529600 cons1,1@int_slack@9 - 8391600 cons1,1@int_slack@10 + [ 6 _e0,0^2
      + 12 _e0,0*_e1,0 + 6 _e0,1^2 + 12 _e0,1*_e1,1 + 6 _e0,2^2 + 12 _e0,2*_

In [20]:
result = cplex.solve(qubo)
print(result)

fval=0.0, x0=0.0, x1=0.0, e0,0=1.0, e0,1=1.0, e0,2=1.0, e1,0=0.0, e1,1=0.0, e1,2=0.0, cons1,0@int_slack@0=1.0, cons1,0@int_slack@1=1.0, cons1,0@int_slack@2=1.0, cons1,0@int_slack@3=1.0, cons1,0@int_slack@4=1.0, cons1,0@int_slack@5=1.0, cons1,0@int_slack@6=1.0, cons1,0@int_slack@7=1.0, cons1,0@int_slack@8=1.0, cons1,0@int_slack@9=1.0, cons1,0@int_slack@10=1.0, cons1,1@int_slack@0=1.0, cons1,1@int_slack@1=1.0, cons1,1@int_slack@2=1.0, cons1,1@int_slack@3=1.0, cons1,1@int_slack@4=1.0, cons1,1@int_slack@5=1.0, cons1,1@int_slack@6=1.0, cons1,1@int_slack@7=1.0, cons1,1@int_slack@8=1.0, cons1,1@int_slack@9=1.0, cons1,1@int_slack@10=1.0, status=SUCCESS


In [21]:
from qiskit import Aer
backend = Aer.get_backend("qasm_simulator")

In [ ]:
qaoa = MinimumEigenOptimizer(QAOA(reps=1, quantum_instance=backend))
result_qaoa = qaoa.solve(qubo)
print(result_qaoa)

<ipython-input-22-1ea5cd56ac63>:1: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.qaoa.QAOA`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.QAOA``. See https://qisk.it/algo_migration for a migration guide.
  qaoa = MinimumEigenOptimizer(QAOA(reps=1, quantum_instance=backend))


In [10]:
# Calculate and print the space utilization
space_utilization = result_qaoa

NameError: ignored